<h1>Ranger: A Fast Implementation of Random Forests</h1>

In [1]:
library(mlr)
source('../utils.r')

set.seed(42)

folder_name = '../../raw_data' 
file_name   = 'german.csv'

Loading required package: ParamHelpers
Warning message:
“replacing previous import ‘BBmisc::isFALSE’ by ‘backports::isFALSE’ when loading ‘mlr’”

<h2>1. Dataprep</h2>

In [2]:
data = read.csv(file=sprintf('%s/%s',folder_name,file_name))

In [3]:
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))
head(data)

NRow: 1000
NCol: 22

X,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
1,A11,6,A34,A43,1169,A65,A75,4,A93,⋯,A121,67,A143,A152,2,A173,good,A192,A201,1
2,A12,48,A32,A43,5951,A61,A73,2,A92,⋯,A121,22,A143,A152,1,A173,good,A191,A201,2
3,A14,12,A34,A46,2096,A61,A74,2,A93,⋯,A121,49,A143,A152,1,A172,bad,A191,A201,1
4,A11,42,A32,A42,7882,A61,A74,2,A93,⋯,A122,45,A143,A153,1,A173,bad,A191,A201,1
5,A11,24,A33,A40,4870,A61,A73,3,A93,⋯,A124,53,A143,A153,2,A173,bad,A191,A201,2
6,A14,36,A32,A46,9055,A65,A73,2,A93,⋯,A124,35,A143,A153,1,A172,bad,A192,A201,1


<p style='color: red'> ATENTION: </p>Specifically in R, in classification problem, target must be set as Factor.

In [4]:
data$V21 = as.factor(data$V21)

MLR works only with features and target, this means that others columns must be dorped.

In [5]:
drops = c()
data  = data[ , !(names(data) %in% drops)]
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))

NRow: 1000
NCol: 22

<h2>2. Modeling</h2>

Function
```R
makeLearner(cl, id = cl, predict.type = "response", predict.threshold = NULL, 
            fix.factors.prediction = FALSE, ..., par.vals = list(), config = list())
```
Param.:

* cl: [character(1)] Class of learner. By convention, all classification learners start with “classif.”. A list of all integrated learners is available on the learners help page < https://mlr-org.github.io/mlr-tutorial/release/html/integrated_learners/ >.
* predict: [character(1)] “response” (= labels) or “prob” (= probabilities and labels by selecting the ones with maximal probability). Default is “response”.
* par.vals: [list] Optional list of named (hyper)parameters. The arguments in ... take precedence over values in this list. We strongly encourage you to use one or the other to pass (hyper)parameters to the learner but not both.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner <br>
Rpart Parameters: https://www.rdocumentation.org/packages/rpart/versions/4.1-11/topics/rpart.control

In [6]:
cl = "classif.ranger"

List all parameters that can be used in this classifier.
The value must be set in par.vals parameter.

In [7]:
getParamSet(cl)

                                      Type  len   Def                    Constr
num.trees                          integer    -   500                  1 to Inf
mtry                               integer    -     -                  1 to Inf
min.node.size                      integer    -     -                  1 to Inf
replace                            logical    -  TRUE                         -
sample.fraction                    numeric    -     -                    0 to 1
split.select.weights         numericvector <NA>     -                    0 to 1
always.split.variables             untyped    -     -                         -
respect.unordered.factors          logical    - FALSE                         -
importance                        discrete    -  none none,impurity,permutation
write.forest                       logical    -  TRUE                         -
scale.permutation.importance       logical    - FALSE                         -
num.threads                        integ

In [8]:
learner = makeLearner(cl = cl
                     , predict.type = "prob"
                     , par.vals = list()
                     )

Function
```R
makeClassifTask(id = deparse(substitute(data)), data, target, weights = NULL, blocking = NULL, 
                positive = NA_character_, fixup.data = "warn", check.data = TRUE)
```
Param.:

* data: [data.frame] A data frame containing the features and target variable(s).
* target: [character(1)] Name of the target variable.
* positive: [character(1)] Positive class for binary classification (otherwise ignored and set to NA). Default is the first factor level of the target attribute.
* fixup.data: [character(1)] Should some basic cleaning up of data be performed? Currently this means removing empty factor levels for the columns. Possible coices are: “no” = Don't do it. “warn” = Do it but warn about it. “quiet” = Do it but keep silent. Default is “warn”.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [9]:
task = makeClassifTask( data = data
                      , target = 'V21'
                      , positive = '2'
                      , fixup.data = 'no'
                      )

Function:
```R
makeResampleDesc(method, predict = "test", ..., stratify = FALSE, stratify.cols = NULL)
```
Param.:

* method: [character(1)] “CV” for cross-validation, “LOO” for leave-one-out, “RepCV” for repeated cross-validation, “Bootstrap” for out-of-bag bootstrap, “Subsample” for subsampling, “Holdout” for holdout.
* predict: What to predict during resampling: “train”, “test” or “both” sets. Default is “test”.
* ... : [any] Further parameters for strategies.
    * iters [integer(1)] Number of iterations, for “CV”, “Subsample” and “Boostrap”.
    * split [numeric(1)] Proportion of training cases for “Holdout” and “Subsample” between 0 and 1. Default is 2/3.
    * reps [integer(1)] Repeats for “RepCV”. Here iters = folds * reps. Default is 10.
    * folds [integer(1)] Folds in the repeated CV for RepCV. Here iters = folds * reps. Default is 10.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeResampleDesc

In [10]:
resample = makeResampleDesc(
    method = "CV",
    iters = 10,
    predict = 'both',
    stratify = FALSE
)

List of performance measures:

Doc.: http://mlr-org.github.io/mlr-tutorial/release/html/measures/

In [11]:
measures = list(mmce #MMCE 
               ,acc  #acuracia
               ,f1   #f1
               ,ppv  #precision
               ,tpr  #recall
               ,auc  #AUC
               ,gini #Gini
               #,timetrain #tempo execucao
               )

Function:
```R
resample(learner, task, resampling, measures, weights = NULL, models = FALSE, extract, 
         keep.pred = TRUE, ..., show.info = getMlrOption("show.info"))
```
Param.:

* learner: [Learner] The learner.
* task: [Task] The task.
* resampling: [ResampleInstance] Resampling strategy.
* measures: [Measure | list of Measure] Performance measure(s) to evaluate. Default is mean misclassification error (mmce)
* weights: [numeric] Optional, non-negative case weight vector to be used during fitting. If given, must be of same length as observations in task and in corresponding order. Overwrites weights specified in the task. By default NULL which means no weights are used unless specified in the task.
* models: [logical(1)] Should all fitted models be returned? Default is FALSE.
* keep.pred: [logical(1)] Keep the prediction data in the pred slot of the result object. If you do many experiments (on larger data sets) these objects might unnecessarily increase object size / mem usage, if you do not really need them. In this case you can set this argument to FALSE. Default is TRUE.
* show.info: [logical(1)] Print verbose output on console? Default is set via configureMlr.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/resample

In [12]:
r = resample(learner = learner
            ,task = task 
            ,resampling = resample 
            ,measures = measures
            #---------------------#
            ,models = TRUE
            ,keep.pred = FALSE
            ,show.info = TRUE
            )

[Resample] cross-validation iter 1: mmce.test.mean=0.25,acc.test.mean=0.75,f1.test.mean=0.324,ppv.test.mean= 0.6,tpr.test.mean=0.222,auc.test.mean=0.739,gini.test.mean=0.478
[Resample] cross-validation iter 2: mmce.test.mean=0.27,acc.test.mean=0.73,f1.test.mean=0.471,ppv.test.mean=0.667,tpr.test.mean=0.364,auc.test.mean=0.744,gini.test.mean=0.488
[Resample] cross-validation iter 3: mmce.test.mean= 0.3,acc.test.mean= 0.7,f1.test.mean=0.423,ppv.test.mean=0.579,tpr.test.mean=0.333,auc.test.mean=0.806,gini.test.mean=0.611
[Resample] cross-validation iter 4: mmce.test.mean=0.26,acc.test.mean=0.74,f1.test.mean=0.458,ppv.test.mean=0.688,tpr.test.mean=0.344,auc.test.mean=0.783,gini.test.mean=0.565
[Resample] cross-validation iter 5: mmce.test.mean=0.23,acc.test.mean=0.77,f1.test.mean=0.531,ppv.test.mean=0.684,tpr.test.mean=0.433,auc.test.mean=0.803,gini.test.mean=0.607
[Resample] cross-validation iter 6: mmce.test.mean=0.23,acc.test.mean=0.77,f1.test.mean=0.511,ppv.test.mean=0.75,tpr.test.mean

<h2>3. Result Analysis</h2>

Train Measures

In [13]:
r$measures.train

iter,mmce,acc,f1,ppv,tpr,auc,gini
1,0.02777778,0.9722222,0.9520154,1.0000000,0.9084249,0.9994275,0.9988549
2,0.03555556,0.9644444,0.9362550,1.0000000,0.8801498,0.9995503,0.9991007
3,0.03333333,0.9666667,0.9404762,1.0000000,0.8876404,0.9996627,0.9993255
4,0.02888889,0.9711111,0.9492188,0.9959016,0.9067164,0.9993092,0.9986185
5,0.03666667,0.9633333,0.9349112,1.0000000,0.8777778,0.9995121,0.9990241
6,0.03333333,0.9666667,0.9409449,1.0000000,0.8884758,0.9995405,0.9990809
7,0.03111111,0.9688889,0.9446640,1.0000000,0.8951311,0.9995089,0.9990178
8,0.03444444,0.9655556,0.9398058,1.0000000,0.8864469,0.9993282,0.9986563
9,0.03111111,0.9688889,0.9463602,1.0000000,0.8981818,0.9995811,0.9991622
10,0.03111111,0.9688889,0.9455253,1.0000000,0.8966790,0.9995776,0.9991552


Test Measures

In [14]:
r$measures.test

iter,mmce,acc,f1,ppv,tpr,auc,gini
1,0.25,0.75,0.3243243,0.6000000,0.2222222,0.7392187,0.4784373
2,0.27,0.73,0.4705882,0.6666667,0.3636364,0.7440072,0.4880145
3,0.30,0.70,0.4230769,0.5789474,0.3333333,0.8055179,0.6110357
4,0.26,0.74,0.4583333,0.6875000,0.3437500,0.7826287,0.5652574
5,0.23,0.77,0.5306122,0.6842105,0.4333333,0.8033333,0.6066667
6,0.23,0.77,0.5106383,0.7500000,0.3870968,0.8078541,0.6157083
7,0.23,0.77,0.6101695,0.6923077,0.5454545,0.8588874,0.7177748
8,0.20,0.80,0.5238095,0.7333333,0.4074074,0.8234399,0.6468798
9,0.20,0.80,0.4444444,0.7272727,0.3200000,0.8250667,0.6501333
10,0.22,0.78,0.5600000,0.6666667,0.4827586,0.7858184,0.5716367


Train Aggregated Result

In [15]:
apply(r$measures.train,2,mean)

iter       mmce        acc         f1        ppv        tpr        auc 
5.50000000 0.03233333 0.96766667 0.94301767 0.99959016 0.89256240 0.99949981 
      gini 
0.99899961

Test Aggregated Result

In [16]:
apply(r$measures.test,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.2390000 0.7610000 0.4855997 0.6786905 0.3838993 0.7975772 0.5951544

Run Time in seconds

In [17]:
r$runtime

[1] 7.42091

<h2>4. Prediction for new data</h2>

Read the data to predict

In [18]:
new.data = read.csv(file=sprintf('%s/%s', folder_name, file_name))

Search for the best model in crossvalidation and use it to score the incoming data

In [19]:
best.model = which.max(r$measures.test$acc)

In [20]:
pred = predict(r$models[[best.model]], newdata = new.data)

Prediction Result

In [21]:
pred

Prediction: 1000 observations
predict.type: prob
threshold: 1=0.50,2=0.50
time: 0.16
  truth    prob.1     prob.2 response
1     1 0.8765603 0.12343968        1
2     2 0.2767151 0.72328492        2
3     1 0.9079040 0.09209603        1
4     1 0.4471286 0.55287143        2
5     2 0.3075119 0.69248810        2
6     1 0.7679397 0.23206032        1
... (1000 rows, 4 cols)


Cast result to data.frame to access the prediction

In [22]:
head(as.data.frame(pred))

truth,prob.1,prob.2,response
1,0.8765603,0.12343968,1
2,0.2767151,0.72328492,2
1,0.9079040,0.09209603,1
1,0.4471286,0.55287143,2
2,0.3075119,0.69248810,2
1,0.7679397,0.23206032,1
